In [1]:
%matplotlib qt
# Set up and load data
import pyxdf
fname = "../data/pilot_1/runs/full_run.xdf"
streams, header = pyxdf.load_xdf(fname)
prediciton_stream = streams[0]["time_series"].T

eeg_streams = streams[1]['time_series'].T


In [2]:
# Prepare for MNE
import mne
import os
import pathlib
import sys
import numpy as np
SCRIPT_DIR = pathlib.Path.cwd()
sys.path.append(os.path.dirname(SCRIPT_DIR))
from continuous_control_bci.util import channel_names
import matplotlib.pyplot as plt
mne.set_log_level('warning') # noqa


In [3]:
eeg_mapping = {name: type for name, type in zip(channel_names[:-8], ["eeg"] * len(channel_names[:-8]))}
emg_mapping = {name: type for name, type in zip(channel_names[-8:-4], ["emg"] * 4)}
eog_mapping = {name: type for name, type in zip(channel_names[-4:], ["eog"] * 4)}



channel_type_mapping = {
    **eeg_mapping,
    **emg_mapping,
    **eog_mapping,
}

wrong_channel_type_mapping = {
    name: type for name, type in zip(channel_names, ["eeg"] * len(channel_names))
}


# The problem is that the EEG is rereferences originally with EMG and EOG. This is silly, but convenient.
# We need a way to deal with this. The easy hack for now is to rereference it in the same way
# Though the correct way is to rereference EEG with EEG and EMG with EMG. Becomes a bit silly that EMG might not work because T7 is bad

raw = mne.io.RawArray(eeg_streams[1:41, :] / 10e5, info=mne.create_info(channel_names, sfreq=2048))

# raw = raw.set_channel_types(wrong_channel_type_mapping)
# raw = raw.set_eeg_reference()
raw = raw.set_channel_types(channel_type_mapping)
raw = raw.set_montage("biosemi32", on_missing='raise')
raw = raw.set_eeg_reference()
# raw = apply_causal_filters(raw, l_eeg=2)
raw = raw.filter(l_freq=0, h_freq=5, picks='eeg')
raw

/var/folders/lm/ht2dkcld0tn4b4c0hqznvyfr0000gn/T/ipykernel_56839/174538854.py:26: RuntimeWarning: The unit for channel(s) AF3, AF4, C3, C4, CP1, CP2, CP5, CP6, Cz, F3, F4, F7, F8, FC1, FC2, FC5, FC6, Fp1, Fp2, Fz, LHEOG, LVEOG, Left-extension, Left-flexion, O1, O2, Oz, P3, P4, P7, P8, PO3, PO4, Pz, RHEOG, Right-extension, Right-flexion, T7, T8, UVEOG has changed from NA to V.
  raw = raw.set_channel_types(channel_type_mapping)


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,35 points
Good channels,"32 EEG, 4 EMG, 4 EOG"
Bad channels,None
EOG channels,"LHEOG, RHEOG, UVEOG, LVEOG"
ECG channels,Not available
Sampling frequency,2048.00 Hz
Highpass,0.00 Hz
Lowpass,5.00 Hz


In [4]:
# ica = mne.preprocessing.ICA(random_state=42)
# ica.fit(raw, picks=['eeg'])
# ica.exclude = ['ICA000', 'ICA001', 'ICA003', 'ICA008', 'ICA009', 'ICA011', 'ICA016', 'ICA021', 'ICA029']
# raw = ica.apply(raw)

In [5]:
# Load events

prediction_timestamps = streams[0]['time_stamps']
prediction_values = streams[0]['time_series']

print(prediction_timestamps.shape)
print(prediction_values[:, 0].shape)
events = np.stack([(prediction_timestamps - streams[1]['time_stamps'][0])*2048, np.zeros(prediction_timestamps.shape), prediction_values[:, 0]]).T

(3733,)
(3733,)


In [6]:
import pickle
with open('../data/pilot_1/emg_model.pkl', 'rb') as f:
    emg_model = pickle.load(f)



In [7]:
raw_emg = raw.copy().pick(['emg'])

In [8]:
raw_emg

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,35 points
Good channels,4 EMG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2048.00 Hz
Highpass,0.00 Hz
Lowpass,5.00 Hz


In [9]:
import scipy
filters = [
    mne.filter.create_filter(raw_emg.get_data(), l_freq=30, h_freq=500, method='iir',
                             phase='forward', sfreq=raw.info['sfreq']),
    mne.filter.create_filter(raw_emg.get_data(), l_freq=51, h_freq=49, method='iir',
                             phase='forward', sfreq=raw.info['sfreq']),
]

raw_data = scipy.signal.sosfilt(filters[0]['sos'],  raw_emg.get_data())
raw_data = scipy.signal.sosfilt(filters[1]['sos'],  raw_data)
raw_emg = mne.io.RawArray(raw_data, raw_emg.info)

In [10]:
raw_emg

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,35 points
Good channels,4 EMG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2048.00 Hz
Highpass,0.00 Hz
Lowpass,5.00 Hz


In [11]:
import numpy as np
prediction_timestamps = streams[0]['time_stamps']
prediction_values = streams[0]['time_series']

print(prediction_timestamps.shape)
print(prediction_values[:, 0].shape)
events = np.stack([(prediction_timestamps - streams[1]['time_stamps'][0])*2048, np.zeros(prediction_timestamps.shape), prediction_values[:, 0]]).T
events = events.astype('int32')

(3733,)
(3733,)


In [12]:
emg_epochs = mne.Epochs(
    raw_emg,
    tmin=-0.2,
    tmax=0.0,
    events=events
)


In [13]:
X_emg = np.abs(emg_epochs.get_data()).mean(axis=2)

In [14]:
y_emg = emg_epochs.events[:, -1]

In [15]:
preds = emg_model.predict(X_emg)
preds[preds==0] = -1
preds[preds==2] = 0
preds[preds==1] = 1.0


agreement = np.mean(preds == y_emg)
print(agreement)

0.8234663809268685


In [16]:
mapping = {
    0: -1.0,  # Left
    1: 1.0,  # Right
    2: 0.0,  # Rest
}


In [17]:

interval = 0.01
epoch_time = 0.2
emg_fine_epochs = mne.make_fixed_length_epochs(
    raw_emg,
    duration=epoch_time,
    overlap=epoch_time - interval,
    reject_by_annotation=False,
)
emg_fine_epochs


Number of events,74679
Events,1: 74679
Time range,0.000 – 0.200 s
Baseline,off


In [18]:
emg_fine_preds = emg_model.predict(np.abs(emg_fine_epochs.get_data()).mean(axis=2))
emg_fine_preds[emg_fine_preds==0] = -1.0
emg_fine_preds[emg_fine_preds==2] = 0.0
emg_fine_preds[emg_fine_preds==1] = 1.0

In [19]:
np.unique(emg_fine_preds, return_counts=True)

(array([-1,  0,  1]), array([37347, 10749, 26583]))

In [20]:
timestamps = np.arange(0, len(raw_emg.times) - epoch_time*2048 , interval * 2048)
timestamps = timestamps + (epoch_time - interval) * 2048

In [21]:

events = np.stack([timestamps, np.zeros(emg_fine_preds.shape), emg_fine_preds]).T.astype('int32')

In [25]:
import itertools
bits = emg_fine_preds
index = 0
starting_points = []
for bit, group in itertools.groupby(bits):
    length = len(list(group))
    if length * interval >= 3.75:
        starting_points.append(events[index, :])
        print(f"{length  * interval} seconds of {events[index, 2]}")
    index += length

# So at the start 
starting_points = np.array(starting_points).astype('int32')
starting_points[:, 0] = starting_points[:, 0] 

17.12 seconds of 0
7.25 seconds of 1
4.19 seconds of 1
8.39 seconds of 1
6.46 seconds of -1
9.19 seconds of 1
10.290000000000001 seconds of -1
4.0 seconds of 1
5.32 seconds of -1
8.66 seconds of -1
7.84 seconds of 1
3.96 seconds of 0
6.96 seconds of 1
10.31 seconds of -1
3.85 seconds of -1
8.950000000000001 seconds of -1
9.99 seconds of 1
6.79 seconds of -1
7.74 seconds of -1
10.42 seconds of 1
11.59 seconds of -1
5.15 seconds of 1
7.4 seconds of -1
6.98 seconds of -1
9.32 seconds of 1
9.620000000000001 seconds of 1
13.17 seconds of -1
4.43 seconds of -1
8.23 seconds of -1
10.01 seconds of 1
8.2 seconds of -1
9.43 seconds of -1
9.85 seconds of 1
10.91 seconds of -1
5.91 seconds of 1
4.37 seconds of -1
10.43 seconds of -1
9.15 seconds of 1
7.84 seconds of 1
12.55 seconds of -1
4.95 seconds of -1
3.81 seconds of 1
8.05 seconds of -1
9.620000000000001 seconds of 1
8.34 seconds of -1
8.36 seconds of -1
9.6 seconds of 1
10.870000000000001 seconds of -1
3.97 seconds of 1
15.67 seconds of -1


In [26]:
epochs = mne.Epochs(
    raw,
    events=starting_points,
    event_id=dict(left=-1, rest=0, right=1),
    tmin=-2,
    tmax=4.75,
)

epochs

Number of events,63
Events,left: 34rest: 2right: 27
Time range,-2.000 – 4.750 s
Baseline,-2.000 – 0.000 s


In [27]:
# MRCP is typically between 5 and 30 μV and therefore easily masked by other brain activities

In [53]:
evokeds = epochs.average(by_event_type=True)

In [54]:
for evoked in evokeds:
    evoked.plot(picks=['C3', 'Cz', 'C4'])
    plt.title(f"{evoked.comment}")

In [52]:
evokeds = dict(
    left=list(epochs["left"].iter_evoked()),
    right=list(epochs["right"].iter_evoked()),
    # rest=list(epochs["rest"].iter_evoked()),
)
mne.viz.plot_compare_evokeds(evokeds, axes='topo', picks=['C3', 'Cz', 'C4'])
plt.show()